## To generate the preProcessed hdf5 files to directly feed to the network

In [1]:
import numpy as np
import os, sys, shutil
import scipy.ndimage as snd
import h5py
import SimpleITK as sitk

##### All the pre-processing functions

In [3]:
def getWeightMap(self, label):
    label = np.argmax(label, axis=3)[0]
    edge = np.float32(morph.binary_dilation(canny(np.float32(label)), selem))
    weight_map = np.zeros(label.shape)
    weight_map[np.where(label>0)] = self.class_weights[1]-1
    # weight_map[np.where(label>0)] = self.getWeightMapValue(label)
    weight_map = weight_map + 1
    weight_map[np.where(edge==1.0)] = 25

    return weight_map[None,:,:]

def downSampleImage(self,image):
    return np.float64(snd.interpolation.zoom(image, 0.5))

def oneHot(self,targets,n_class = 6):
    axis = targets.ndim
    ret_val = (np.arange(n_class) == np.expand_dims(targets,axis = axis)).astype(int)
    return ret_val

def loadDicomVolume(self,file_path):
    reader = sitk.ImageSeriesReader()
    reader.SetOutputPixelType(sitk.sitkFloat32)
    dicom_names = reader.GetGDCMSeriesFileNames(file_path)
    reader.SetFileNames(dicom_names)
    self.itk_image = reader.Execute()
    image_vol = sitk.GetArrayFromImage(self.itk_image)
    image_vol = np.transpose(image_vol,(1,2,0))
    return np.float32(image_vol)

def HU_window(self, in_arr):
    return np.clip(in_arr, -100, 400)

def histogram_equalization(self, arr):
    nbr_bins = 256
    imhist, bins = np.histogram(arr.flatten(), nbr_bins, normed = True)
    cdf = imhist.cumsum()
    cdf = 255 * cdf / cdf[-1]
    original_shape = arr.shape
    arr = np.interp(arr.flatten(), bins[:-1], cdf)
    out_arr = arr.reshape(original_shape)
    return out_arr

def normalize(self, x):
    x = np.float32(x)
    min_val = np.min(x)
    max_val = np.max(x)
    ret_val = (x - min_val) / (max_val - min_val)
    return ret_val

def makeInput(image):
    return image[None,:,:,None]

##### Generate all the filepaths of the volumes and corresponding segmenataitons

In [ ]:
root = ''
dest = ''
mode = 'train'
all_fls = [os.path.join(root,mode,f) for f in os.listdir(os.path.join(root,mode))]

vols = []
segs = []

for each in all_fls:
    if 'volume-' in each:
        vols.append(each)
    elif 'segmentation-' in each:
        segs.append(each)

vols.sort()
segs.sort()

In [ ]:
for vol_path in vols:
    vol_img = sitk.ReadImage(vol_path)
    vol = sitk.GetArrayFromImage(vol_img)
    for i in range()